In [1]:
# Importing all the libraries
import re
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from pdfminer.high_level import extract_text
import difflib
import textdistance
import spacy
from spacy.matcher import Matcher
from summarizer import Summarizer
import warnings
warnings.filterwarnings('default')
import nltk
nltk.download('punkt')
import glob
import os
import fitz

C:\Users\pawan\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pawan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Define the directory and folder name 
df_skill = pd.read_excel('C:\\Users\\pawan\\Desktop\\data scientist skill.xlsx',sheet_name='Skills')
lst_skills = []
for skill in df_skill.Skills:
    lst_skills.append(skill.strip())
    
df_education = pd.read_excel('C:\\Users\\pawan\\Desktop\\data scientist skill.xlsx',sheet_name='Education')
lst_education = []
for edu in df_education.Education_List:
    lst_education.append(edu.strip())
df_university = pd.read_excel('C:\\Users\\pawan\\Desktop\\data scientist skill.xlsx',sheet_name='University')
lst_university = []
for edu in df_university.University:
    lst_university.append(edu)


In [3]:
# Defining the phone number pattern
pattern = [
    {"ORTH": "+"},
    {"ORTH": "91"},
    {"SHAPE": "dddddddddd"}
]

In [4]:
# Extracting the text data from each resume.
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

In [5]:
# Extracting contact number from each resume
def extract_contact_number_from_resume(text):
    contact_number = None
    # regex pattern to find a best match contact number
    pattern = r"\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"
    match = re.search(pattern, text)
    if match:
        contact_number = match.group()
   
    return contact_number

In [6]:
# Extracting email address from each resume
def extract_email_from_resume(text):
    email = None
    # To find best match email address
    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    match = re.search(pattern, text)
    if match:
        email = match.group()
   
    return email

In [7]:
# Extracting skills from each resume
def extract_skills_from_resume(text, skills_list):
    skills = []
    for skill in skills_list:
        pattern = r"\b{}\b".format(re.escape(skill))
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            skills.append(skill)
        
    return skills 

In [8]:
# Extracting education from each resume
def extract_education_from_resume(text):
    education = []
    # Use regex pattern to find education information
    pattern = r"(?i)(?:Bsc|\bB\.\w+|\bM\.\w+|\Diploma\.D\.\w+|\bPh\.D\.\w+|\bBachelor(?:'s)?|\bPostgraduate(?:'s)?|\bMaster(?:'s)?|\bPh\.D)\s(?:\w+\s)*\w+"
    matches = re.findall(pattern, text)
    for match in matches:
        education.append(match.strip())
        
    for match_edu in lst_education:
        eduk=difflib.get_close_matches(text, match_edu, cutoff=0.75)
        education.append(eduk)
        education = [ele for ele in education if ele != []]
    
    return education

In [9]:
# Extracting name from each resume
def extract_name(resume_text):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)

    # Define name patterns
    patterns = [
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name, Middle name, and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]  # First name, Middle name, Middle name, and Last name
        # Add more patterns as needed
    ]

    for pattern in patterns:
        matcher.add('NAME', patterns=[pattern])

    doc = nlp(resume_text)
    matches = matcher(doc)

    for match_id, start, end in matches:
        span = doc[start:end]
        
        return span.text

In [10]:
# # Extracting university from each resume
def extract_university_from_resume(text, university_list):
    university = []
    for uni in university_list:
        un=difflib.get_close_matches(text, uni, cutoff=0.80) # Match percentage of resume text and user requirement text.
        university.append(un)
    university = [ele for ele in university if ele != []]
    
    return university

In [11]:
# Calling the main function.
if __name__ == '__main__':
    
    #reading user input text file where all requirement mentioned for resume screening.
    resume_text_data = open(r'C:\Users\pawan\Desktop\folder\resume_comparitive_file.txt', "r")
    
    # Creating a Dataframe with different columns
    detail_df=pd.DataFrame(columns=['Name','Contact_Number' ,'Education' ,'University','Skill_Matchs','Email','Resume_Summary','Data_similarity'])
    #------------------------------------1-----------------------------------#
    
    # Definig the directory 
    directory ='C://Users//pawan//Desktop//FOLDER//'
    
    # Defining all the list to append data extracted from different functions
    found_skill_lst=[]
    found_university_lst=[]
    found_education_lst=[]
    found_name_lst=[]
    found_contact_number_lst=[]
    found_email_lst=[]
    clear_resume_summary_lst=[]
    found_summary_similarity_lst=[]
    
    for val,filename in enumerate(os.listdir(directory)):
        if filename.endswith('.pdf'):                            # Reading all file in folder have extension '.pdf'.
            with open(os.path.join(directory, filename)) as f:
                filename_updated = directory+filename            # Defining the directory and folder of all resume
        text = extract_text_from_pdf(filename_updated)
        
  
        # List of predefined skills
    
        skills_list = lst_skills
        extracted_skills = extract_skills_from_resume(text, skills_list)
        found_skill_lst.append(','.join(extracted_skills))
            #     ------------------------------------2-----------------------------------#
            
        # List of predefined university
        university_list = lst_university
        extracted_university = extract_university_from_resume(text, lst_university)
        found_university_lst.append(','.join(extracted_university))
            #     ------------------------------------3-----------------------------------#

        # List of predefined education    
        extracted_education = extract_education_from_resume(text)
        found_education_lst.append(','.join(extracted_education))
        #     ------------------------------------4-----------------------------------#
        #Extracting name and appending it to list
        name = extract_name(text)
        found_name_lst.append(name)
        #     ------------------------------------5-----------------------------------#

        contact_number = extract_contact_number_from_resume(text)
        found_contact_number_lst.append(contact_number)
        #     ------------------------------------6-----------------------------------#

        email = extract_email_from_resume(text)
        found_email_lst.append(email)
         #     ------------------------------------7-----------------------------------#
            
        # Using BERT model to create summry of each resume into 400 words as maximum limit.
        model = Summarizer()
        result = model(text, max_length=400)
        full = ''.join(result)
        cleanString = re.sub('\W+',' ', full)
        clear_resume_summary_lst.append(cleanString)
        
        
    # Putting all the list values to its respective column value.
    
    detail_df['Skill_Matchs'] =  pd.Series(found_skill_lst)
    detail_df['University'] =  pd.Series(found_university_lst)
    detail_df['Education'] =  pd.Series(found_education_lst)
    detail_df['Name'] =  pd.Series(found_name_lst)
    detail_df['Contact_Number'] =  pd.Series(found_contact_number_lst)
    detail_df['Email'] =  pd.Series(found_email_lst)
    detail_df['Resume_Summary'] =  pd.Series(clear_resume_summary_lst)
    #--------------------------------8--------------------------------------
    

<>:65: DeprecationWarning: invalid escape sequence \W
<>:65: DeprecationWarning: invalid escape sequence \W
C:\Users\pawan\AppData\Local\Temp\ipykernel_3700\248798750.py:65: DeprecationWarning: invalid escape sequence \W
  cleanString = re.sub('\W+',' ', full)
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForS

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.pre

In [12]:
detail_df.head()

,Name,Contact_Number,Education,University,Skill_Matchs,Email,Resume_Summary,Data_similarity
0,Abhay Kumar,None,Bachelor of Engineering,,"python,modeling,development,spark,deep learnin...",abhayownsthis@gmail.com,Abhay Kumar Lead Data Scientist Computer Visio...,NaN
1,Abhijit Manepatil,None,"Master in Computer Engineering and 3,Master of...",,"sql,python,modeling,deep learning,machine lear...",None,Abhijit Manepatil Data Scientist with Master i...,NaN
2,AJAY KUMAR,None,Master Degree in Computer Science from IIITB w...,,"python,scala,development,hadoop,spark,machine ...",ajaychouksey75@gmail.com,In depth knowledge and experience in statistic...,NaN
3,Aniket Kashyap,9691316944,Bachelor of Engineering,,"sql,python,development,sql server,deep learnin...",monu.aniket@gmail.com,Aniket Kashyap Software Developer Data Scienti...,NaN
4,Ashly Lau,None,,,"python,development,machine learning",ashlylau@gmail.com,,NaN


## Resume Screening

In [13]:
resume_screening = pd.read_csv('C://Users//pawan//Desktop//FOLDER/check_parse.csv')
path = 'C://Users//pawan//Desktop//FOLDER//'
for filename in glob.glob(os.path.join(path, '*.txt')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        d=f.read()
for summary in resume_screening.Resume_Summary:
    found_summary_similarity_lst.append(textdistance.cosine(d, str(summary)))
detail_df['Data_similarity'] =  pd.Series(found_summary_similarity_lst)

#---------------------------------------------

nlp = spacy.load('en_core_web_sm') # loads english NLP model (small)
pdfFiles = []
nlp_clear_resume_summary_lst =[]
directory =path
for filename in os.listdir(directory):
    if filename.endswith('.pdf'):
        os.chdir('C://Users//pawan//Desktop//FOLDER//')
        pdfFileObj = open(filename, 'rb')
        with fitz.open(pdfFileObj) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
            doc = nlp(text) # process the sentence with the nlp model and retrieve entities
            model = Summarizer()
            nlp_result = model(doc, max_length=400)
            nlp_full = ''.join(nlp_result)
            nlp_cleanString = re.sub('\W+',' ', nlp_full)
            pdfFiles.append(nlp_cleanString)
detail_df['NLP_Based_Summary'] =  pd.Series(pdfFiles)

<>:28: DeprecationWarning: invalid escape sequence \W
<>:28: DeprecationWarning: invalid escape sequence \W
C:\Users\pawan\AppData\Local\Temp\ipykernel_3700\3870715529.py:28: DeprecationWarning: invalid escape sequence \W
  nlp_cleanString = re.sub('\W+',' ', nlp_full)
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a

C:\Users\pawan\AppData\Local\Temp\ipykernel_3700\3870715529.py:19: ResourceWarning: unclosed file <_io.BufferedReader name='Himanshu-Tyagi.pdf'>
  pdfFileObj = open(filename, 'rb')
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\User

In [14]:
# Remove the redundent data if any for safe side.
detail_df = detail_df.drop_duplicates(keep='last')
detail_df

,Name,Contact_Number,Education,University,Skill_Matchs,Email,Resume_Summary,Data_similarity,NLP_Based_Summary
0,Abhay Kumar,None,Bachelor of Engineering,,"python,modeling,development,spark,deep learnin...",abhayownsthis@gmail.com,Abhay Kumar Lead Data Scientist Computer Visio...,0.839653,Abhay Kumar Lead Data Scientist Computer Visio...
1,Abhijit Manepatil,None,"Master in Computer Engineering and 3,Master of...",,"sql,python,modeling,deep learning,machine lear...",None,Abhijit Manepatil Data Scientist with Master i...,0.569463,Abhijit Manepatil Data Scientist with Master i...
2,AJAY KUMAR,None,Master Degree in Computer Science from IIITB w...,,"python,scala,development,hadoop,spark,machine ...",ajaychouksey75@gmail.com,In depth knowledge and experience in statistic...,0.851100,Thorough understanding of big data technologi...
3,Aniket Kashyap,9691316944,Bachelor of Engineering,,"sql,python,development,sql server,deep learnin...",monu.aniket@gmail.com,Aniket Kashyap Software Developer Data Scienti...,0.798867,Aniket Kashyap Software Developer Data Scienti...
4,Ashly Lau,None,,,"python,development,machine learning",ashlylau@gmail.com,,0.046225,Ashly Lau ashlylau gmail com linkedin in ashly...
5,Ashly Lau,None,,,"python,development,machine learning",ashlylau@gmail.com,,0.046225,ACHAL AGARWAL agarwal achal94 gmail com 91 981...
6,Junior Developer,None,Bachelor of Technology,,"data analytics,sql,python,sql server,hadoop,ma...",agarwal.achal94@gmail.com,ACHAL AGARWAL agarwal achal94 gmail com 91 981...,0.728342,SUMMARY CONTACT Data Scientist with 2 years of...
7,DEEPJYOTI DUTTA,91-897-109-3779,Bachelor of Computer,,"sql,python,sql server,deep learning,machine le...",deepjyoti.dutta@outlook.com,I have also done some internal projects as Dat...,0.805401,Himanshu Tyagi Pune Maharashtra Email me on In...
8,Himanshu Tyagi,None,,,"data mining and statistical analysis,sql,pytho...",None,Himanshu Tyagi Pune Maharashtra Email me on In...,0.635227,Jalpa Dave Data Scientist Elegant Microweb Tec...
9,Jalpa Dave,None,,,"data analytics,sql,python,sql server,statistic...",None,Jalpa Dave Data Scientist Elegant Microweb Tec...,0.714560,Page 1 of 3 Curriculum Vitae Lavajiit Singh Mo...


In [15]:
# Save the dataframe to csv format.
detail_df.to_csv('C://Users//pawan//Desktop//FOLDER/check_parse.csv')